In [ ]:
import sys
!"{sys.executable}" -m pip install --upgrade pyarrow

In [ ]:
import pandas as pd
import pyarrow
import sys

print("Python:", sys.executable)
print("pandas:", pd.__version__)
print("pyarrow:", pyarrow.__version__)

In [ ]:
from pathlib import Path
import pandas as pd

ROOT = Path("/Users/constan/Desktop")
DATA_DIR = ROOT / "data" / "cicids2017_raw"

parquets = sorted(DATA_DIR.glob("*.parquet"))
print("Parquet files found:", len(parquets))

dfs = []
for p in parquets:
    df = pd.read_parquet(p)

    name = p.name.lower()
    if "benign" in name:
        df["label"] = "benign"
        df["attack_type"] = "none"
    else:
        df["label"] = "classical_attack"
        df["attack_type"] = p.stem.replace("-no-metadata", "")

    dfs.append(df)

df_raw = pd.concat(dfs, ignore_index=True)
print("Total rows loaded:", len(df_raw))
df_raw.head()
